＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝整理＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝　　

迁移学习常用思路：  　　

①最简单：若待分类数据集和预训练数据集(imagenet)非常接近，可以考虑直接用预训练模型进行预测（泛化能力弱）　　

②第二简单：获取预训练模型的bottleneck_feature作为新的feature,然后用新的feature训练简单的分类器，如logistic_regressioN  
（This method is more likely to generalize well to a broader range of problems, including problems featuring classes absent from ImageNet.）

③稍复杂：手动复制预训练模型架构，导入权重，然后pop掉某些层，加入自己的层（比如pop掉最后一个全连接层换成自己的全连接层）

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝注意＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝  
①一些常用的pandas函数:pd.groupby(['']).count.sort_values(by='xx',ascending=False),pd.isin()  
②划分训练测试集的简易写法  
rnd = np.random.random(len(labels))  
train_idx = rnd<0.8  
valid_idx = rnd>=0.8  
③keras中的图像读取以及预处理工具:  
keras.preprocessing.image工具箱中提供了load_image(name,target_size)和img_to_array等函数    
· 各个预训练模型如keras.applications.xception提供了preprocess_input函数，接收一个扩充维度之后的图像(ndarray形式！)，将图像处理成可以直接输入预训练模型的形式  
④使用预训练模型时要注意：  
·　若使用bottleneck feature,设置(include=True)  
·　一种简便的方法，对预训练模型设置(poolin='avg')，这样使用模型对数据集进行预测之后，得到的最后一层输出（提取出的bottleneck_feature）shape为2-D形式。可以直接送入机器学习的分类器，如LogisticRegression或者svm等

# 首先学习一个Kaggler的方法，脚本最后使用自己的方法进行50分类练习

一、准备数据集： 使用频率最高的16个类别　　

注意：每张图像名对应一个label,存放于label.csv中

In [196]:
import keras
import os
from os.path import join, exists
import pandas as pd
import numpy as np
from keras.preprocessing import image
from tqdm import tqdm
import keras
from keras.applications.vgg16 import preprocess_input,VGG16
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from keras.applications.xception import Xception
from keras.applications import xception
import matplotlib.pyplot as plt

In [2]:
DATA_DIR = os.getcwd()
INPUT_SIZE = 224
NUM_CLASSES = 16
NUM_CLASSES = 16

In [3]:
labels = pd.read_csv(join(DATA_DIR,'labels.csv'))
sample_submission = pd.read_csv('sample_submission.csv')

提交样例格式：每张图像名称对应所有类别的输出概率

In [4]:
sample_submission.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


初始的labels.csv概览

In [5]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [6]:
print(len(os.listdir(join(DATA_DIR,'train'))),len(labels))
print(len(os.listdir(join(DATA_DIR, 'test'))), len(sample_submission))
#训练集和测试集的样本数

10222 10222
10357 10357


取出Top16类别————使用pd.groupby()将对象通过某一个column分组，然后计算各个分组个数，排序

In [7]:
labels.groupby('breed').count().sort_values(by='id',ascending=False).head(NUM_CLASSES).index

Index(['scottish_deerhound', 'maltese_dog', 'afghan_hound', 'entlebucher',
       'bernese_mountain_dog', 'shih-tzu', 'great_pyrenees', 'pomeranian',
       'basenji', 'samoyed', 'airedale', 'tibetan_terrier', 'leonberg',
       'cairn', 'beagle', 'japanese_spaniel'],
      dtype='object', name='breed')

In [8]:
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id',
                                                                       ascending=False).head(NUM_CLASSES).index)

In [9]:
labels['breed'].isin(selected_breed_list)[:10]

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8     True
9     True
Name: breed, dtype: bool

In [10]:
labels = labels[labels['breed'].isin(selected_breed_list)]
len(labels)

1777

In [11]:
labels['target']=1 #增加一列全1作为value

In [12]:
labels.head()

,id,breed,target
8,003df8b8a8b05244b1d920bb6cf451f9,basenji,1
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound,1
12,00693b8bc2470375cc744a6391d397ec,maltese_dog,1
29,00bee065dcec471f26394855c5c2f3de,cairn,1
48,013f8fdf6d638c7bb042f5f17e8a9fdc,tibetan_terrier,1


In [13]:
labels.pivot(index='id',columns='breed',values='target')[:5]

breed,afghan_hound,airedale,basenji,beagle,bernese_mountain_dog,cairn,entlebucher,great_pyrenees,japanese_spaniel,leonberg,maltese_dog,pomeranian,samoyed,scottish_deerhound,shih-tzu,tibetan_terrier
id,,,,,,,,,,,,,,,,
003df8b8a8b05244b1d920bb6cf451f9,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0042188c895a2f14ef64a918ed9c7b64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
00693b8bc2470375cc744a6391d397ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
00bee065dcec471f26394855c5c2f3de,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
013f8fdf6d638c7bb042f5f17e8a9fdc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [14]:
#重新设置一列数字index
labels_pivot = labels.pivot(index='id',columns='breed',values='target').reset_index().fillna(0)
labels_pivot[:5]

breed,id,afghan_hound,airedale,basenji,beagle,bernese_mountain_dog,cairn,entlebucher,great_pyrenees,japanese_spaniel,leonberg,maltese_dog,pomeranian,samoyed,scottish_deerhound,shih-tzu,tibetan_terrier
0,003df8b8a8b05244b1d920bb6cf451f9,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0042188c895a2f14ef64a918ed9c7b64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,00693b8bc2470375cc744a6391d397ec,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,00bee065dcec471f26394855c5c2f3de,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,013f8fdf6d638c7bb042f5f17e8a9fdc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
#选出80%作为train set
rnd = np.random.random(len(labels))
train_idx = rnd<0.8
valid_idx = rnd>=0.8
y_train = labels_pivot[selected_breed_list].values #我们想要的label形式
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [17]:
y_val = y_train[valid_idx]
y_train=y_train[train_idx]

准备X_train 和 X_val

In [21]:
labels['id'][:5]

8     003df8b8a8b05244b1d920bb6cf451f9
9     0042188c895a2f14ef64a918ed9c7b64
12    00693b8bc2470375cc744a6391d397ec
29    00bee065dcec471f26394855c5c2f3de
48    013f8fdf6d638c7bb042f5f17e8a9fdc
Name: id, dtype: object

In [67]:
def read_img(img_id, train_or_test, size):
    img = image.load_img(join(DATA_DIR,'train','{}.jpg'.format(img_id)),target_size=size)
    img = image.img_to_array(img)
    return img

In [68]:

print(labels['id'][8])
img=read_img(labels['id'][8],'train',(INPUT_SIZE,INPUT_SIZE))

003df8b8a8b05244b1d920bb6cf451f9


In [98]:
X = np.zeros((len(labels),INPUT_SIZE,INPUT_SIZE,3),dtype='float32')
for i,img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id,'train',(INPUT_SIZE,INPUT_SIZE))
    x = preprocess_input(np.array([img]))  #VGG16中的函数，将输入nparray直接处理成网络输入可用形式 
    X[i]=x
print('Train Image shape: {0}   size:　 {1:,}'.format(X.shape, X.size))                 

1777it [00:03, 497.86it/s]

Train Image shape: (1777, 224, 224, 3)   size:　 267,488,256


In [97]:
#对比：先全部取出来再preprocess_input。注意不要这样操作。按照keras官方文档教程应该对每张图像扩充维度之后进行preprocess input

# X = np.zeros((len(labels),INPUT_SIZE,INPUT_SIZE,3),dtype='float32')
# for i,img_id in tqdm(enumerate(labels['id'])):
#     img = read_img(img_id,'train',(INPUT_SIZE,INPUT_SIZE))
# #     x = preprocess_input(np.array([img]))  #VGG16中的函数，将输入nparray直接处理成网络输入可用形式 
#     X[i]=x
# X1 = preprocess_input(X)
# print('Train Image shape: {0}   size:　 {1:,}'.format(X.shape, X.size))    

1777it [00:03, 546.90it/s]


Train Image shape: (1777, 224, 224, 3)   size:　 267,488,256


In [90]:
X_train = X[train_idx]
X_val = X[valid_idx]
print(X_train.shape,  y_train.shape, X_val.shape, y_val.shape)

(1432, 224, 224, 3) (1432, 16) (345, 224, 224, 3) (345, 16)


## 1.使用VGG16进行预训练（方法②）

提取vgg_16的bottleneck feature

In [104]:
vgg_bottleneck = VGG16(weights='imagenet',include_top=False,pooling='avg')
train_vgg_bf = vgg_bottleneck.predict(X_train,batch_size = 32)
val_vgg_bf = vgg_bottleneck.predict(X_val,batch_size = 32)

58892288/58889256 [==============================] - 78s 1us/step


In [163]:
train_vgg_bf.shape #为什么bottleneck_feature尺寸是512?不是7*7*512? 答案：最后输出的是global_average_pooling层的输出。
# 见vgg_bottleneck.summery()

(1432, 512)

In [125]:
logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs')
logreg.fit(train_vgg_bf,np.argmax(y_train,axis=1))
val_probs=logreg.predict_proba(val_vgg_bf) 
val_preds=logreg.predict(val_vgg_bf)

In [150]:
print('Validation VGG LogLoss {}'.format(log_loss(y_val, val_probs)))
print('Validation VGG accuracy',accuracy_score(val_preds,np.argmax(y_val, axis=1)))

Validation VGG LogLoss 0.3770159855860719
Validation VGG accuracy 0.8898550724637682


In [151]:
from sklearn.svm import SVC
svm_clf = SVC(kernel='linear')
svm_clf.fit(train_vgg_bf,np.argmax(y_train,axis=1))
pred_svm = svm_clf.predict(val_vgg_bf)
accuracy_score(pred_svm,np.argmax(y_val,1))
print('Validation VGG SVM accuracy', _ )

Validation VGG SVM accuracy 0.9043478260869565


## 2.使用Xception进行预训练


In [173]:
Xcpt_bottleneck = Xception(weights='imagenet',include_top=False,pooling='avg') 

注意：这里选择pooling='avg'很重要，这样使用bottleneck模型对数据集进行predict，最后的输出是global_average_pooling的输出，shape为(None,2048)，这样的shape可以直接送入sklearn中的分类器，无需任何处理

In [159]:
vgg_bottleneck.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [170]:
Xcpt_bottleneck.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [175]:
train_xcpt_bf.shape #输出和vgg不同

(1432, 2048)

注意：使用Xception model时要重新对数据集进行预处理。其预处理方法和vgg不同

In [225]:
X=np.zeros((len(labels),299,299,3),dtype='float32')
for i,id in tqdm(enumerate(labels['id'])):
    img = read_img(id,'train',(299,299))
    img = xception.preprocess_input(np.array([img])) #注意先扩展维度
    X[i] = img
X_train = X[train_idx]
X_val = X[valid_idx]
print(X_train.shape,X_val.shape)    

1777it [00:03, 481.54it/s]


(1432, 299, 299, 3) (345, 299, 299, 3)


In [228]:
Xcpt_bottleneck = Xception(weights='imagenet',include_top=False,pooling='avg') 
print('Model loaded....')
train_xcpt_bf = Xcpt_bottleneck.predict(X_train,batch_size=32,verbose=1)
val_xcpt_bf = Xcpt_bottleneck.predict(X_val,batch_size=32,verbose=1)
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42)
print('Using machine learning classifier to fit dataset...')
logreg.fit(train_xcpt_bf, np.argmax(y_train,axis=1))
pred_logreg = logreg.predict(val_xcpt_bf)
score=accuracy_score(pred_logreg,np.argmax(y_val,axis=1))
print('使用Xception预训练模型+logreg准确率为',score)

Model loaded....
345/345 [==============================] - 2s 7ms/step
Using machine learning classifier to fit dataset...
使用Xception预训练模型+logreg准确率为 0.991304347826087


# 练习: 使用预训练模型进行50分类　准确率0.946

#### 使用sklearn中的预处理函数代替上面pandas的one-hot及text attributes转换环节;
#### 使用python列表推倒式代替pd.isin函数

In [1]:
import pandas as pd
import numpy as np
from keras.applications import xception
import os

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA_DIR = os.getcwd()
NUM_CLASSES = 50
labels = pd.read_csv('labels.csv')

In [3]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


选取频率最高的50个类别

In [4]:
selected_breed = list(labels.groupby('breed').count().sort_values(by='id',ascending=False)[:NUM_CLASSES].index)

In [5]:
len(selected_breed)

50

从labels中去除其余类别对应的instance————关键:isin函数  
两种方法：一种pandas，一种python（使用列表推倒式）

In [6]:
# selected_labels_filtered = list(filter(lambda x: x in selected_breed, labels['breed']))
print(np.sum([ x in selected_breed for x in labels['breed']]))
selected_labels = labels[[ x in selected_breed for x in labels['breed']]]

4890


In [7]:
selected_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


In [8]:
labels['breed'][0] in selected_breed

True

In [9]:
selected_labels_pd=labels[labels['breed'].isin(selected_breed)]
len(selected_labels_pd)

4890

In [10]:
selected_labels_pd.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


现在精简之后的pd只有50个类别。需要对每个instance的标签进行转换：字符串映射到数字并且使用one-hot编码  
这里考虑使用sklearn.preprocessing

In [11]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
#注意LabelEncoder的bug，不要放在pipeline中使用
label_encoder = LabelEncoder()
txt_label = label_encoder.fit_transform(selected_labels['breed'])
one_hot_encoder = OneHotEncoder(sparse=False)
mapped_label = one_hot_encoder.fit_transform(txt_label.reshape(-1,1))

使用刚学的方法进行train,validation split

In [29]:
TRAIN_RATIO = 0.8
y = mapped_label
rnd = np.random.random(len(selected_labels))
train_ind = rnd<TRAIN_RATIO
val_ind = rnd>=TRAIN_RATIO
y_train = y[train_ind]
y_val = y[val_ind]
print(y_train.shape,y_val.shape)
y_train_compact = np.argmax(y_train,axis=1)
y_val_compact = np.argmax(y_val,axis=1)

(3868, 50) (1022, 50)


In [58]:
np.sum(train_ind)


4890

使用Xception model作为预训练模型

In [13]:
from keras.preprocessing import image
def read_img(img_id,train_or_test,size):
    img = image.load_img(join(DATA_DIR,train_or_test,'{}.jpg'.format(img_id)),target_size=size)
    img = image.img_to_array(img)
    img = xception.preprocess_input(np.array([img]))
    return img        

In [14]:
from tqdm import tqdm
from os.path import join
INPUT_SIZE=299
X = np.zeros((len(selected_labels),299,299,3),dtype='float32')
for i, id in tqdm(enumerate(selected_labels['id'])):
    img = read_img(id,'train',(INPUT_SIZE,INPUT_SIZE))
    X[i] = img

4890it [00:11, 408.31it/s]


In [62]:
X_train = X[train_ind]
X_val = X[val_ind]

使用预训练模型进行特征提取

In [64]:
xception_bottleneck = xception.Xception(weights='imagenet',include_top=False,pooling='avg')
#bottleneck feature
train_bf = xception_bottleneck.predict(X_train,verbose=1)
val_bf = xception_bottleneck.predict(X_val,verbose=1)

1022/1022 [==============================] - 7s 7ms/step


In [65]:
len(train_ind)

4890

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score
logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs')
logreg.fit(train_bf,y_train_compact)
predictions = logreg.predict(val_bf)
print('准确率：　',accuracy_score(predictions,y_val_compact))
print('f1 Score: ',f1_score(y_val_compact,predictions,average='macro'))

准确率：　 0.9461839530332681
f1 Score:  0.9449219531868012
